In [ ]:
import harp
import numpy as np
from pathlib import Path
import os
import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal
import copy

In [ ]:
base_path = Path('/home/ikharitonov/Desktop/')
dataset_name = '2024-05-10T14-04-37'

In [ ]:
def get_register_object(register_number, harp_board='h1'):
    
    h1_reader = harp.create_reader(f'h1-device.yml', epoch=harp.REFERENCE_EPOCH)
    h2_reader = harp.create_reader(f'h2-device.yml', epoch=harp.REFERENCE_EPOCH)
    reference_dict = {
        'h1': {
            32: h1_reader.Cam0Event,
            33: h1_reader.Cam1Event,
            38: h1_reader.StartAndStop,
            46: h1_reader.OpticalTrackingRead
        },
        'h2': {
            38: h2_reader.Encoder,
            39: h2_reader.AnalogInput,
            42: h2_reader.ImmediatePulses
        }
    }
    return reference_dict[harp_board][register_number]

In [ ]:
optical_stream_df = utils.load(get_register_object(46, 'h1'), base_path/dataset_name/'HarpDataH1')

plt.figure()
plt.plot(optical_stream_df['OpticalTrackingRead0X'], label='X')
plt.plot(optical_stream_df['OpticalTrackingRead0Y'], label='Y')
plt.plot(optical_stream_df['OpticalTrackingRead0Brightness'], label='Brightness')
plt.plot(optical_stream_df['OpticalTrackingRead1X'], label='X2')
plt.plot(optical_stream_df['OpticalTrackingRead1Y'], label='Y2')
plt.plot(optical_stream_df['OpticalTrackingRead1Brightness'], label='Brightness2')
plt.legend()
plt.show()

In [ ]:
optical_stream_x = optical_stream_df['OpticalTrackingRead0X']
print(len(optical_stream_x))
print(optical_stream_x)

## Checking inter-timestamp intervals (optic flow_x)

In [ ]:
print('All unique element-wise differences between timestamps (in nanoseconds)', np.unique(np.diff(optical_stream_x.index).astype(np.uint32)))

In [ ]:
plt.hist(np.diff(optical_stream_x.index).astype(np.uint32)* (10**-9), bins=50)
plt.xlabel('Inter-timestamp interval (seconds)')

In [ ]:
(9984000+10016000)* (10**-9)

## Comparing clocked and non-clocked datasets

In [ ]:
h1_reader = harp.create_reader('h1-device.yml', epoch=harp.REFERENCE_EPOCH)
h2_reader = harp.create_reader('h2-device.yml', epoch=harp.REFERENCE_EPOCH)
session_data_reader = utils.SessionData()

base_path = '/home/ikharitonov/Desktop/'

dataset_name = '2024-05-10T14-04-37'
nonclocked_h1_46 = utils.load(h1_reader.OpticalTrackingRead, base_path + dataset_name + '/HarpDataH1/')
nonclocked_h2_42 = utils.load(h2_reader.ImmediatePulses, base_path + dataset_name + '/HarpDataH2/')
nonclocked_h2_38 = utils.load(h2_reader.Encoder, base_path + dataset_name + '/HarpDataH2/')

dataset_name = '2024-06-26T13-51-40_gain_A'
clocked_h1_46 = utils.load(h1_reader.OpticalTrackingRead, base_path + dataset_name + '/HarpDataH1/')
clocked_h2_42 = utils.load(h2_reader.ImmediatePulses, base_path + dataset_name + '/HarpDataH2/')
clocked_h2_38 = utils.load(h2_reader.Encoder, base_path + dataset_name + '/HarpDataH2/')

In [ ]:
print(nonclocked_h1_46.index[0])
print(nonclocked_h1_46.index[1])
print(nonclocked_h1_46.index[2])
print(nonclocked_h1_46.index[3])
print(nonclocked_h1_46.index[4])
print(nonclocked_h1_46.index[5])
print(nonclocked_h1_46.index[6])
print(nonclocked_h1_46.index[7])

In [ ]:
print(nonclocked_h2_38.index[0])
print(nonclocked_h2_38.index[1])
print(nonclocked_h2_38.index[2])
print(nonclocked_h2_38.index[3])
print(nonclocked_h2_38.index[4])
print(nonclocked_h2_38.index[5])
print(np.diff(nonclocked_h2_38.index[:6]))

In [ ]:
clocked_h1_46.index[0]

In [ ]:
clocked_h2_42.index[0]

In [ ]:
clocked_h2_42.index[1]

In [ ]:
clocked_h2_38.index[0]

In [ ]:
clocked_h2_38.index[1]

In [ ]:
print(clocked_h1_46.index[0])
print(clocked_h1_46.index[1])
print(clocked_h1_46.index[2])
print(clocked_h1_46.index[3])
print(clocked_h1_46.index[4])
print(clocked_h1_46.index[5])
print(np.diff(clocked_h1_46.index[:6]))

In [ ]:
print(clocked_h2_38.index[0])
print(clocked_h2_38.index[1])
print(clocked_h2_38.index[2])
print(clocked_h2_38.index[3])
print(clocked_h2_38.index[4])
print(clocked_h2_38.index[5])
print(np.diff(clocked_h2_38.index[:6]))

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=2, figsize=(10,10))

ax[0][0].plot(nonclocked_h1_46)
ax[0][0].set_title('nonclocked_h1_46')

ax[1][0].plot(nonclocked_h2_42)
ax[1][0].set_title('nonclocked_h2_42')

ax[0][1].plot(clocked_h1_46)
ax[0][1].set_title('clocked_h1_46')

ax[1][1].plot(clocked_h2_42)
ax[1][1].set_title('clocked_h2_42')

ax[2][0].plot(nonclocked_h2_38)
ax[2][0].set_title('nonclocked_h2_38')

ax[2][1].plot(clocked_h2_38)
ax[2][1].set_title('clocked_h2_38')

plt.tight_layout()
plt.show()

## Checking particular timestamp

In [ ]:
timestamp = '1904-01-13 05:45:19.910240'

In [ ]:
clocked_h1_46[clocked_h1_46.index==timestamp]

In [ ]:
clocked_h2_42[clocked_h2_42.index==timestamp]

### Lomb–Scargle periodogram

Lomb-Scargle periodogram was developed by Lomb [Lomb, N.R., 1976] and further extended by Scargle [Scargle, J.D., 1982] to find, and test the significance of weak periodic signals with uneven temporal sampling. Lomb–Scargle periodogram is a method that allows efﬁcient computation of a Fourier-like power spectrum estimator from unevenly sampled data, resulting in an intuitive means of determining the period of oscillation.
https://www.earthinversion.com/techniques/how-to-deal-with-irregular-sparse-data-set/

In [ ]:
f = np.linspace(0.001, 10**20, 1000)
pgram = signal.lombscargle(clocked_h2_42.index.to_numpy(), clocked_h2_42.values[:,0], f, normalize=True)
pgram2 = signal.lombscargle((clocked_h2_42.index - clocked_h2_42.index[0]).total_seconds().to_numpy(), clocked_h2_42.values[:,0], f, normalize=True)

In [ ]:
plt.plot(f,pgram)

In [ ]:
plt.plot(f,pgram2)